## Alpha test COBE EEG

- Import data
- Re-code trigger codes
- Segment data
- FFT

### First, run FFT on the whole data-set, without segmenting trials

In [1]:
% matplotlib inline

In [2]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pylab
import os

In [3]:
file = 'Trial3.2016-08-29_data_export_from_OpenViBe.csv'

pathin = '/Users/ethan/Dropbox/Documents/Research/Projects_current/alpha_test/COBE_EEG_fft/'

os.chdir(pathin)

df = pd.read_csv(file, sep=',')

#files = os.listdir(pathin)

#print(files)
#for s, val in enumerate(files):
#    print(val)
#data = np.loadtxt(fname = file, delimiter = ',')

In [4]:
df.head(n=5)

,Time (s),Channel 1,Channel 2,Channel 3,Channel 4,Channel 5,Channel 6,Channel 7,Channel 8,Channel 9,Channel 10,Channel 11,Channel 12,Channel 13,Channel 14,Channel 15,Channel 16,Trigger line,Sampling Rate
0,0.000000,31073,768,2147343744,-284380,-1142870,2147343744,-2811848,2147343744,-8018963,-1382663,-7209147,461289,-929298,-211929,514893,289949,0,512.0
1,0.001953,-20810,768,2147226496,-312499,-1185205,2147226496,-2887760,2147226496,-8105692,-1451050,-7302532,455687,-974374,-268679,501995,289543,0,NaN
2,0.003906,-54145,768,2147141504,-329707,-1213877,2147141504,-2940432,2147141504,-8165736,-1497299,-7368458,453206,-1006460,-306649,497305,289851,0,NaN
3,0.005859,179411,768,2147419520,-173945,-1008798,2147419520,-2699314,2147419520,-7915805,-1252841,-7109418,510934,-790162,-71392,611696,308983,0,NaN
4,0.007813,431080,768,-2147190528,-25864,-799844,-2147190528,-2418509,-2147190528,-7617925,-979429,-6793012,555812,-566286,180232,704305,320584,0,NaN


### First, check frequencies from 1-60. 
Our data is heavily contaminated with line noise, so there should be a big peak at around 50 Hz, and the FFT shows this

In [ ]:

data = df.as_matrix()
xstart = 1
xend = 60
pp = PdfPages('all_data_freq1-'+str(xend)+'.pdf')

chans = [1,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
#chans = [9]

# get time-stamp data
t = data[:,0]

for s in chans:
    C = data[:,s]

    y = C
    Y    = np.fft.fft(y)
    freq = np.fft.fftfreq(len(y), t[1] - t[0])
    
    # remove DC (non-varying) component
    freq = freq[1:]
    Y = np.abs(Y[1:])

    pylab.figure()
    pylab.plot( freq, Y )
    plt.xlabel('Channel '+str(s))
    
    #pp.savefig()
    plt.xlim(xstart,xend)
    #plt.ylim(0,0.2)
    
#pp.close()



### Now, let's zoom in to frequencies below 50 Hz
Channels 8 and 9 look to be the ones with most signal, so we'll focus on those

In [ ]:
import scipy as sp
import numpy as np
data = df.as_matrix()
xstart = 1
xend = 40
pp = PdfPages('all_data_freq1-'+str(xend)+'.pdf')

#chans = [1,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
chans = [9]

# get time-stamp data
t = data[:,0]

for s in chans:
    C = data[:,s]

    y = C
    Y    = np.fft.fft(y)
    freq = np.fft.fftfreq(len(y), t[1] - t[0])
    
    # remove DC (non-varying) component
    freq = freq[1:]
    Y = np.abs(Y[1:])

    pylab.figure()
    pylab.plot( freq, Y )
    plt.xlabel('Channel '+str(s))
    
    #pp.savefig()
    plt.xlim(xstart,xend)
    #plt.ylim(0,0.2)
    
#pp.close()

### Let's try to filter the line noise out

In [ ]:
import scipy.signal as signal

xstart = 1
xend = 60


chans = [9]

# get time-stamp data
t = data[:,0]

# plot fft of raw data
for s in chans:
    y = data[:,s]

    # run fft
    Y    = np.fft.fft(y)
    freq = np.fft.fftfreq(len(y), t[1] - t[0])
    
    # remove DC (non-varying) component
    freq = freq[1:]
    Y = np.abs(Y[1:])
    
    plt.figure()
    plt.plot( freq, Y )
    plt.xlim(xstart, xend)
    plt.xlabel('raw data')

# plot fft of low-pass filtered data
for s in chans:
    y = data[:,s]
    
    # fit butterworth low-pass filter
    b, a = signal.butter(4, 0.08, 'low')
    output_signal = signal.filtfilt(b, a, y)
    
    # run fft
    y = output_signal
    Y    = np.fft.fft(y)
    freq = np.fft.fftfreq(len(y), t[1] - t[0])
    
    # remove DC (non-varying) component
    freq = freq[1:]
    Y = np.abs(Y[1:])
    
    plt.figure()
    plt.plot( freq, Y )
    plt.xlim(xstart, xend)
    plt.xlabel('Low-pass filtered')
    
    # fit butterworth high-pass filter
    #b,a = signal.butter(1, .1, 'high')
    #output_signal = signal.filtfilt(b, a, output_signal)

#plot fft of 50 Hz notch-filtered data
for s in chans:    
    # fit 50 Hz notch filter
    y = data[:,s]

    fs = 512
    band_width = np.array([49, 51.0])
    b, a = signal.butter(2,band_width/(fs / 2), 'bandstop')
    output_signal = signal.filtfilt(b, a, y)

    # run fft
    y = output_signal
    Y    = np.fft.fft(y)
    freq = np.fft.fftfreq(len(y), t[1] - t[0])

    # remove DC (non-varying) component
    freq = freq[1:]
    Y = np.abs(Y[1:])

    plt.figure()
    plt.plot( freq, Y )
    plt.xlim(xstart, xend)
    plt.xlabel('Notch filtered')

# plot fft of low-pass and notch-filtered data
for s in chans:
    fs = 512
    band_width = np.array([49, 51.0])
    b, a = signal.butter(2,band_width/(fs / 2), 'bandstop')
    output_signal = signal.filtfilt(b, a, output_signal)

    # fit butterworth low-pass filter
    b, a = signal.butter(4, 0.08, 'low')
    output_signal = signal.filtfilt(b, a, output_signal)    
    
    # run fft
    y = output_signal
    Y    = np.fft.fft(y)
    freq = np.fft.fftfreq(len(y), t[1] - t[0])

    # remove DC (non-varying) component
    freq = freq[1:]
    Y = np.abs(Y[1:])

    plt.figure()
    plt.plot( freq, Y )
    plt.xlim(xstart, xend)
    plt.xlabel('Low-pass and notch filtered')


 

### Now, let's try segmenting the data into events

In [ ]:
file = 'Trial3.2016-08-29_data_export_from_OpenViBe.csv'

pathin = '/Users/ethan/Dropbox/Documents/Research/Projects_current/alpha_test/COBE_EEG_fft/'

os.chdir(pathin)

df = pd.read_csv(file, sep=',')
Seg1 = df.loc[df['Channel 2'] == 768]
Seg2 = df.loc[df['Channel 2'] == 770]

In [ ]:
Seg1.head(n=5)

In [ ]:
Seg2.head(n=5)

In [ ]:
import numpy as np
data = Seg1.as_matrix()
xstart = 1
xend = 40
pp = PdfPages('seg_data_freq1-'+str(xend)+'.pdf')

#chans = [1,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
chans = [9]

# get time-stamp data
t = data[1:3000,0]

for s in chans:
    C = data[3000:6000,s]

    y = C
    Y    = np.fft.fft(y)
    freq = np.fft.fftfreq(len(y), t[1] - t[0])
    
    # remove DC (non-varying) component
    freq = freq[1:]
    Y = np.abs(Y[1:])

    pylab.figure()
    pylab.plot( freq, Y )
    plt.xlabel('Channel '+str(s))
    
    #pp.savefig()
    plt.xlim(xstart,xend)
    #plt.ylim(0,0.2)
    
#pp.close()


### Next steps
One problem is that the data have not been properly segmented. For some reason, we have many more "768" event codes than "770" event codes, so it is hard to know what belongs to what event (eyes open vs. eyes closed). I might try a moving window analysis on the whole time-series, just for fun. I think the next bit causes a memory problem, so careful! Needs to be fixed before running again!!

In [ ]:
% matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal

file = 'Trial3.2016-08-29_data_export_from_OpenViBe.csv'

pathin = '/Users/ethan/Dropbox/Documents/Research/Projects_current/alpha_test/COBE_EEG_fft/'

os.chdir(pathin)

df = pd.read_csv(file, sep=',')
data = df.as_matrix()

xstart = 1
xend = 60


chans = [9]

# get time-stamp data
t = data[:,0]


len(t)
def round_down(num, divisor):
    return num - (num%divisor)
x = round_down(len(t),1000)
y = data[0:x,9]
win = np.split(y, 2)


# plot fft of low-pass filtered data
for seg in win:
    for i, s in enumerate(chans):
        y = seg

        # fit butterworth low-pass filter
        b, a = signal.butter(4, 0.08, 'low')
        output_signal = signal.filtfilt(b, a, y)

        # run fft
        y = output_signal
        Y    = np.fft.fft(y)
        freq = np.fft.fftfreq(len(y), t[1] - t[0])

        # remove DC (non-varying) component
        freq = freq[1:]
        Y = np.abs(Y[1:])

        plt.figure()
        plt.plot( freq, Y )
        plt.xlim(xstart, xend)
        plt.xlabel('Low-pass filtered')

In [ ]:
len(t)
def round_down(num, divisor):
    return num - (num%divisor)
x = round_down(len(t),100)
y = data[0:x,9]
win = np.split(y, 500)
len(win[1])